In [2]:
import pprint
import os

# Create VectorStore

In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [29]:
model_name = "hkunlp/instructor-large"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
from langchain.llms import CTransformers
model = CTransformers(model="TheBloke/Llama-2-7B-GGUF",model_file="llama-2-7b.Q4_K_M.gguf", model_type="llama",config={"gpu_layers":50})

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
pprint.pprint(model("what is retrieval augmented generation"))

('\n'
 'Retrieval Augmented Generation (RAG) is a new approach to the problem of '
 'generating text from a given input. It combines two techniques: retrieval '
 'and generation. In RAG, we first retrieve relevant documents from a large '
 'corpus using a search engine. We then use these retrieved documents as '
 'inputs for a neural network that generates new text based on the input.\n'
 'The advantage of this approach is that it can generate more diverse and '
 'higher-quality output than traditional methods such as GPT-3 or BERT. '
 'Additionally, RAG is less computationally expensive since it does not '
 'require training a large model from scratch.\n'
 'There are several different ways to implement RAG, but one popular method is '
 'to use a transformer-based architecture. In this case, the input documents '
 'are fed into an encoder layer of the transformer network and then passed '
 'through a decoder layer that generates new text based on the input.\n'
 'Another way to implemen

# Load PDF files

In [16]:
from langchain.document_loaders import PyPDFLoader
import pandas as pd

In [17]:
DATA_DIR = '../data/gsk/'
df = pd.read_pickle(os.path.join(DATA_DIR, 'gsk.pkl'))

In [22]:
pages = []
for i, row in df.iterrows():
    pages.extend(PyPDFLoader(os.path.join(DATA_DIR,row['file_name']+'.pdf')).load())

In [24]:
len(pages)

148

In [35]:
all_splits[0]

Document(page_content='For media and investors only  \n  \n \nPress release  1 \n \nIssued: 6 December  2023, London  UK \n \nNew global analysis  across five cities  shows inequities in \nadult immunisation  uptake,  signalling need to redesign \nlocal and national policy interventions  \n• New IQVIA Institute for Human Data Science report , funded by GSK , explore s \nthe role of social and structural determinants of health  to identify enablers and \nbarriers to vaccine uptake  in Bangkok, B russels , Chicago,  Manchester  and \nNew York City  \n• As one example, in both New York City and Chicago, household income, \neducation and median house value are highly correlated with immunisation  \nrates across all vaccines studied    \n• Findings to be presented today at the Global Coalition on Aging’s Silver \nEconomy Forum 2023   \n \nGSK plc (LSE/NYSE: GSK), in collaboration  with the Global Coalition on Aging (GCOA), announced a new report', metadata={'source': '../data/gsk/25444525-9

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(pages)

In [26]:
len(all_splits)

542

In [30]:
vectorstore = FAISS.from_documents(all_splits, hf)

In [34]:
vectorstore.similarity_search("strategic vaccine partnership in China")

[Document(page_content='company by revenue , has a track record  of driving access to innovative vaccines in China . By bringing together the \nscale and expertise  of the two companies , the s trategic partnership will significantly extend the  availability  of \nShingrix , supporting the rapid expan sion of  access to the vaccine  and future potential indications.  \n \nSet to start on 1 January 2024, Zhifei  will have exclusive rights to import and distribute Shingrix  in China – focusing  \non promoting the vaccine  through  its extensive  service network , which cover s more than 30,000 vaccination points  \nacross the country . In partnership, GSK  - as the license holder for the product  - will co -promote Shingrix  by raising  \nawar eness of the importance of shingles vaccination amongst healthcare professionals  within community health \ncentres  and hospital settings.  \n \nIn August  2023 , GSK announced positive data from the ZOSTER -076 phase IV trial  showing that Shingr

In [19]:
loader = PyPDFLoader("/home/ubuntu/oulas/text_summarization_pipeline/data/gsk/42b46554-a9ee-4010-8137-e748deb65269.pdf")
pages = loader.load_and_split()

In [13]:
len(pages)

3

In [8]:
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())